In [72]:
import pandas as pd
from pathlib import Path

In [73]:
# PROJECT_ROOT = Path(r"C:\Users\dipes\OneDrive\Documents\Machine Learning\Python\Structural Optimization\aero_surrogate")

# TEMPLATE_PATH = PROJECT_ROOT / "config" / "baselines" / "template.dat"
# DESIGNS_PATH  = PROJECT_ROOT / "windows" / "datasets" / "designs_6dv.csv"
# OUTPUT_ROOT   = PROJECT_ROOT / "wsl" / "fea_cases"   # where case_0001, case_0002, ... will live

# # Make sure output root exists
# OUTPUT_ROOT.mkdir(parents=True, exist_ok=True)



In [90]:


# 1) Paths
PROJECT_ROOT = Path("C:/Users/dipes/OneDrive/Documents/Machine Learning/Python/Structural Optimization/aero_surrogate")
REPO_ROOT = Path("C:/Users/dipes/OneDrive/Documents/Machine Learning/Python/Structural Optimization/aero_surrogate/wsl/tblade3")

CASE7_DIR = REPO_ROOT / "inputs" / "OpenCSM" / "Case7-28-8" / "rotor"
TEMPLATE_PATH = PROJECT_ROOT / "config" / "baselines" / "template.dat"

print("CASE7_DIR:", CASE7_DIR)
print("Exists:", CASE7_DIR.exists())

# 2) Find the original .dat file that contains the sweep/lean block
dat_files = list(CASE7_DIR.glob("*.dat"))
print("Found .dat files:")
for f in dat_files:
    print(" -", f.name)

if not dat_files:
    raise RuntimeError("No .dat files found in Case7-28-8 directory")

# Try each .dat and pick the one with 'Control points for sweep'
source_file = None
for f in dat_files:
    text = f.read_text()
    if "Control points for sweep" in text and "Control points for lean" in text:
        source_file = f
        break

if source_file is None:
    raise RuntimeError("Could not find a .dat file containing sweep/lean control points")

print("Using source .dat file:", source_file)

# 3) Copy its content into template.dat in your project
content = source_file.read_text()
print("Source length:", len(content))

TEMPLATE_PATH.parent.mkdir(parents=True, exist_ok=True)
TEMPLATE_PATH.write_text(content)

print("Wrote template.dat at:", TEMPLATE_PATH)
print("Template size now:", TEMPLATE_PATH.stat().st_size)


CASE7_DIR: C:\Users\dipes\OneDrive\Documents\Machine Learning\Python\Structural Optimization\aero_surrogate\wsl\tblade3\inputs\OpenCSM\Case7-28-8\rotor
Exists: True
Found .dat files:
 - 3dbgbinput.2.dat
 - spancontrolinputs.2.dat
Using source .dat file: C:\Users\dipes\OneDrive\Documents\Machine Learning\Python\Structural Optimization\aero_surrogate\wsl\tblade3\inputs\OpenCSM\Case7-28-8\rotor\3dbgbinput.2.dat
Source length: 254149
Wrote template.dat at: C:\Users\dipes\OneDrive\Documents\Machine Learning\Python\Structural Optimization\aero_surrogate\config\baselines\template.dat
Template size now: 258953


In [94]:
DESIGNS_PATH  = PROJECT_ROOT / "windows" / "datasets" / "designs_6dv.csv"
OUTPUT_ROOT   = PROJECT_ROOT / "wsl" / "fea_cases"   # where case_0001, case_0002, ... will live

# Make sure output root exists
OUTPUT_ROOT.mkdir(parents=True, exist_ok=True)

In [95]:
# Check if baseline baseline exists and list its contents
BAS = PROJECT_ROOT / "config" / "baselines"
print("Baselines exists:", BAS.exists())

for f in BAS.iterdir():
    print(" -", f.name)

Baselines exists: True
 - template.dat


In [96]:
with open(TEMPLATE_PATH, "r") as f:
    template_text = f.read()
    
print("Exists:", TEMPLATE_PATH.exists())

print("Loaded template from:", TEMPLATE_PATH)
print("Template length:", len(template_text))
print("Template preview:\n", template_text[:56000])

Exists: True
Loaded template from: C:\Users\dipes\OneDrive\Documents\Machine Learning\Python\Structural Optimization\aero_surrogate\config\baselines\template.dat
Template length: 254037
Template preview:
  Input parameters (version 1.31_TEST)
 Case7-28-8
 Blade row #:
           2
 Number of blades in this row:
          16
 Blade Scaling factor (mm):
   152.40000000000001     
 Number of streamlines:
          21
 Angles in the input file (0=Beta_z (default),1=Beta_r):
            0  inci_dev_spline
 Airfoil camber defined by curvature control (0=no,1=yes ;spanwise_spline= spancontrolinputs file):
            1  spanwise_spline
 Airfoil Thickness distribution (0=Wennerstrom,1=Spline,2=Spline + sharp TE):
           5
 Airfoil Thickness multiplier (0=no,1=yes):
           0
 Airfoil LE defined by spline (0=no,1=yes):
           0
 Non-dimensional Actual chord (0=no,1=yes,2=spline):
           2
 True lean and sweep (0=no,1=yes):
           0
 Clustering control (0=uniform,1=sine,2=expo

In [97]:
# --------- LOAD DESIGNS ---------

df_designs = pd.read_csv(DESIGNS_PATH)
print("Loaded designs:", DESIGNS_PATH)
print(df_designs.head())


Loaded designs: C:\Users\dipes\OneDrive\Documents\Machine Learning\Python\Structural Optimization\aero_surrogate\windows\datasets\designs_6dv.csv
   case     dms25     dms50    dms100     dth25     dth50    dth100
0     1  0.006415  0.003371 -0.008791 -0.014232  0.027969  0.030008
1     2  0.007083  0.002481  0.003858 -0.047817 -0.007457  0.022691
2     3 -0.005576 -0.000522  0.004704  0.019591 -0.034518 -0.042688
3     4  0.009478 -0.000909 -0.001784 -0.024785 -0.012657  0.012036
4     5 -0.002052  0.007320  0.008102  0.041319  0.024949 -0.009561


In [98]:
required_cols = ["case", "dms25", "dms50", "dms100", "dth25", "dth50", "dth100"]
missing = [c for c in required_cols if c not in df_designs.columns]
if missing:
    raise ValueError(f"Missing required columns in designs CSV: {missing}")


In [99]:
# Expected columns: case, dms25, dms50, dms100, dth25, dth50, dth100

required_cols = ["case", "dms25", "dms50", "dms100", "dth25", "dth50", "dth100"]
missing = [c for c in required_cols if c not in df_designs.columns]
if missing:
    raise ValueError(f"Missing required columns in designs CSV: {missing}")


In [100]:
# --------- FUNCTION TO FILL ONE TEMPLATE ---------

def fill_template(dvs: dict) -> str:
    """
    dvs: dict like {"dms25": 0.003, "dth25": -0.02, ...}
    Returns: string with all %...% placeholders replaced by numeric values
    """
    text = template_text
    for name, value in dvs.items():
        placeholder = f"%{name}%"
        # Format floats nicely; you can tweak precision if needed
        text = text.replace(placeholder, f"{value:.15f}")
    return text

In [101]:
# --------- GENERATE INPUTS FOR EACH CASE ---------

for idx, row in df_designs.iterrows():
    case_id = int(row["case"])
    
    # Build DV dict for this row
    dvs = {
        "dms25": row["dms25"],
        "dms50": row["dms50"],
        "dms100": row["dms100"],
        "dth25": row["dth25"],
        "dth50": row["dth50"],
        "dth100": row["dth100"],
    }
    
    # Fill template
    filled_text = fill_template(dvs)
    
    # Case directory: case_0001, case_0002, ...
    case_dir = OUTPUT_ROOT / f"case_{case_id:04d}"
    case_dir.mkdir(parents=True, exist_ok=True)
    
    # Output file: input.dat
    out_path = case_dir / "input.dat"
    with open(out_path, "w") as f:
        f.write(filled_text)
    
    if case_id <= 3:  # print first few as sanity check
        print(f"Written case {case_id} to {out_path}")

print("DONE: generated T-Blade3 input .dat files for all cases.")

Written case 1 to C:\Users\dipes\OneDrive\Documents\Machine Learning\Python\Structural Optimization\aero_surrogate\wsl\fea_cases\case_0001\input.dat
Written case 2 to C:\Users\dipes\OneDrive\Documents\Machine Learning\Python\Structural Optimization\aero_surrogate\wsl\fea_cases\case_0002\input.dat
Written case 3 to C:\Users\dipes\OneDrive\Documents\Machine Learning\Python\Structural Optimization\aero_surrogate\wsl\fea_cases\case_0003\input.dat
DONE: generated T-Blade3 input .dat files for all cases.
